In [41]:
import numpy as np
import os
import nibabel as nib

from scipy import ndimage
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [70]:
def create_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model

def deserialize(serialized_example):
    feature_map = {
        'x_shape': tf.io.FixedLenFeature([3], dtype=tf.int64),
        'x': tf.io.FixedLenFeature([], dtype=tf.string),
        'y': tf.io.FixedLenFeature([], dtype=tf.string)
    }

    example = tf.io.parse_single_example(serialized_example, feature_map)

    x_shape = example['x_shape']
    x_bytes = example['x']
    y_bytes = example['y']
    
    x = tf.io.decode_raw(x_bytes, out_type=tf.float64)
    x = tf.reshape(x, x_shape)
    x = tf.cast(x, tf.float32)
    y = tf.io.parse_tensor(y_bytes, out_type=tf.float32)
    y = tf.cast(y, tf.int64)
    return x, y

In [71]:
train_data = tf.data.TFRecordDataset('training_dataset.tfrecord')
for record in train_data:
    x, y = deserialize(record)
    print(x.shape)
    print(y)


(128, 128, 64)
tf.Tensor([1 0 0 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([1 0 0 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([1 0 0 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([0 1 0 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([0 0 1 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([0 0 1 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([1 0 0 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([0 0 0 1], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([0 0 1 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([1 0 0 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([1 0 0 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([1 0 0 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([1 0 0 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([1 0 0 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([1 0 0 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor([1 0 0 0], shape=(4,), dtype=int64)
(128, 128, 64)
tf.Tensor

KeyboardInterrupt: 

In [43]:
train_data = tf.data.TFRecordDataset('training_dataset.tfrecord')
train_data = train_data.map(deserialize)


for x,y in train_data:
    print(y)

InvalidArgumentError: Input to DecodeRaw has length 26 that is not a multiple of 8, the size of int64
	 [[{{node DecodeRaw_1}}]] [Op:IteratorGetNext]

In [40]:

model = create_model()

initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), metrics='accuracy')

batch_size = 1
epochs = 2
validation_split = 0.1

with tf.device("GPU"):
    for x,y in train_data:
        model.fit(
            x=x,
            y=y,
            epochs=epochs,
            shuffle=True,
            verbose=2
        )



ValueError: Data cardinality is ambiguous:
  x sizes: 128
  y sizes: 1
Make sure all arrays contain the same number of samples.